<a href="https://colab.research.google.com/github/Julialunna/Adversarial_Attack_Defense/blob/main/Adversarial_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from flautim.pytorch.Model import Model


class Model(Model):
    def __init__(self, context, num_classes=10, **kwargs):
        super(Model, self).__init__(context, name="LeNet5", **kwargs)

        # entrada esperada -> imagens preta e branca 1x28x28
        #caso tenha cores (RGB)
        #self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1)
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)

        # Após convs/pools, o tamanho típico de saída é 16x4x4 (para entrada 28x28)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

        #testar se fica melhor com o dropout
        self.dropout = nn.Dropout(p=0.1)

    @staticmethod
    def build_lenet5(num_classes=10):
        """
        Retorna uma instância da LeNet-5 pura (sem wrapper do Model).
        """
        class LeNet5(nn.Module):
            def __init__(self):
                super().__init__()
                self.conv1 = nn.Conv2d(1, 6, 5)
                self.pool = nn.AvgPool2d(2)
                self.conv2 = nn.Conv2d(6, 16, 5)
                self.fc1 = nn.Linear(16 * 4 * 4, 120)
                self.fc2 = nn.Linear(120, 84)
                self.fc3 = nn.Linear(84, num_classes)

            def forward(self, x):
                x = F.relu(self.conv1(x))
                x = self.pool(x)
                x = F.relu(self.conv2(x))
                x = self.pool(x)
                x = x.view(x.size(0), -1)
                x = F.relu(self.fc1(x))
                x = F.relu(self.fc2(x))
                x = self.fc3(x)
                return x

        return LeNet5()

    def forward(self, x):
        # Fluxo da LeNet-5
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:

#Chamar flautim.pytorch.centralized.Experiment para experimento centralizado
from flautim.pytorch.centralized.Experiment import Experiment

import flautim2 as fl
import numpy as np
import torch
import time

class Experiment(Experiment):
    def __init__(self, model, dataset, context, **kwargs):
        super(Experiment, self).__init__(model, dataset, context, **kwargs)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)
        self.epochs = kwargs.get('epochs', 30)


    def training_loop(self, data_loader):
        self.model.train()
        error_loss = 0.0
        yhat, y_real = [], []

        for X, y in data_loader:
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()

            error_loss += loss.cpu().item()
            _, predicted = torch.max(outputs.data, 1)
            yhat.append(predicted.detach().cpu())
            y_real.append(y.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)

        return float(error_loss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

    def validation_loop(self, data_loader):
        error_loss = 0.0
        self.model.eval()

        with torch.no_grad():
            for X, y in data_loader:
                outputs = self.model(X)
                error_loss += self.criterion(outputs, y).item()
                _, predicted = torch.max(outputs.data, 1)
                yhat.append(predicted.detach().cpu())
            y_real.append(y.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)

        return float(error_loss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

In [ ]:
#Determinar qual dataset para aí sim eu mexer aqui, vamo começar com qual?
from flautim2.pytorch.Dataset import Dataset
import torch
import copy

class IRISDataset(Dataset):

    def __init__(self, file, **kwargs):
        super(IRISDataset, self).__init__(name = "IRIS", **kwargs)

        # Defina o que são features e targets
        self.features = file.iloc[:, 0:4].values
        self.target = file.iloc[:, 4].values

        # Número de amostras para teste
        self.test_size = int(0.2 * len(file))

        # Defina o tipo do tensor de entrada e de saída.
        self.xdtype = torch.float32
        self.ydtype = torch.int64

        # batch_size
        self.batch_size = 10

        # shuffle
        self.shuffle = True

        # num_workers
        self.num_workers = 1

    def train(self) -> Dataset:
        # Separação das amostras para treino
        self.features = self.features[:-self.test_size]
        self.target = self.target[:-self.test_size]
        return copy.deepcopy(self)

    def validation(self) -> Dataset:
        # Separação das amostras para validação
        self.features = self.features[-self.test_size:]
        self.target = self.target[-self.test_size:]
        return copy.deepcopy(self)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.LongTensor([self.target[idx]])